In [1]:
%load_ext kedro.extras.extensions.ipython
%reload_kedro /Users/yeungadrian/Documents/repo/TimeSeries

2022-06-05 20:53:19,943 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-06-05 20:53:35,697 - root - INFO - ** Kedro project timeseries
2022-06-05 20:53:35,699 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-06-05 20:53:36,031 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-06-05 20:53:36,106 - root - INFO - ** Kedro project timeseries
2022-06-05 20:53:36,108 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


## Libraries

In [ ]:
from pydantic import BaseModel
from typing import List, Optional
import numpy as np
from typing import TypeVar

In [ ]:
PandasDataFrame = TypeVar('pandas.core.frame.DataFrame')

## Data

In [4]:
price_df = catalog.load('fund_prices')
price_df

2022-06-05 21:13:19,568 - kedro.io.data_catalog - INFO - Loading data from `fund_prices` (ParquetDataSet)...


,date,ABMD,ATVI,AMD,AMZN,AAPL,DTE,EBAY
0,2020-12-31,324.200,92.850000,91.710,3256.93,132.690000,121.410000,50.250000
1,2020-12-30,323.920,91.580000,92.290,3285.85,133.720000,119.660000,50.550000
2,2020-12-29,320.930,91.370000,90.620,3322.00,134.870000,119.240000,50.860000
3,2020-12-28,312.910,91.430000,91.600,3283.96,136.690000,119.230000,50.240000
4,2020-12-24,303.410,90.960000,91.810,3172.69,131.970000,119.720000,50.120000
...,...,...,...,...,...,...,...,...
5322,1999-11-05,11.315,1.148739,10.625,64.94,0.680100,13.626682,6.904808
5323,1999-11-04,10.470,1.091975,10.315,63.06,0.643981,13.360490,6.733871
5324,1999-11-03,10.250,1.083012,10.655,65.81,0.627654,13.626682,6.583654
5325,1999-11-02,10.315,1.064339,10.280,66.44,0.618028,13.626682,6.801210


## Backtesting Portfolio

In [17]:
class Portfolio(BaseModel):
    fund_codes: List[str]
    fund_weights: List[str]
    date: str
    amount: float
    df: PandasDataFrame

    def backtest_funds(
        self
    ):
        number_funds = len(self.fund_codes)

        fund_diagonal = np.zeros((number_funds, number_funds))
        np.fill_diagonal(fund_diagonal, self.fund_weights)

        result = self.df[self.fund_codes].dot(fund_diagonal)
        result.columns = self.fund_codes

        result["portfolio"] = result.sum(axis=1)

        return result


In [18]:
external_data = {
    "fund_codes": ["AMD", "AMZN", "AAPL"],
    "fund_weights": [0.3,0.4,0.3],
    "date": "2018-12-31",
    "amount": 1000,
    "df": price_df
}

In [19]:
mock_portfolio = Portfolio(**external_data)

In [20]:
mock_portfolio.backtest_funds()

,AMD,AMZN,AAPL,portfolio
0,27.5130,1302.772,39.807000,1370.092000
1,27.6870,1314.340,40.116000,1382.143000
2,27.1860,1328.800,40.461000,1396.447000
3,27.4800,1313.584,41.007000,1382.071000
4,27.5430,1269.076,39.591000,1336.210000
...,...,...,...,...
5322,3.1875,25.976,0.204030,29.367530
5323,3.0945,25.224,0.193194,28.511694
5324,3.1965,26.324,0.188296,29.708796
5325,3.0840,26.576,0.185408,29.845408
